In [2]:
from datasets import load_dataset
from pathlib import Path
from pprint import pprint
from datasets import load_dataset
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import openai
from openai import APIError
import os
import json
import re
import numpy as np
from sklearn.cluster import KMeans
from pprint import pprint
from pathlib import Path
import tiktoken


# Load the .env file
load_dotenv()

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY



In [33]:
loaded_table = pq.read_table('datasets/msmarco.parquet')
df = loaded_table.to_pandas()

In [34]:
df

,id,query,passage_text
0,0,da li ljudska kosa zaustavlja veverice,[Hranimo naše veverice u dvorištu tokom jeseni...
1,1,Koji su benefiti fosilnih goriva?,[Najveća prednost korišćenja fosilnih goriva j...
2,2,šta je apotema,[Apotema. Apotema pravilnog mnogougla je segme...
3,3,average cost for custom canopy,"[Čestitamo! Pronašli ste BuyShade.com, vaš izv..."
4,4,šta je hardver u računaru,"[Hardver se najbolje opisuje kao uređaj, poput..."
5,5,edi logistics definition,[Kao i mnoge druge rane informacione tehnologi...
6,6,why should recreational marijuana be illegal,[Zašto bi rekreativna marihuana trebalo da bud...
7,7,U koju klasu spadaju paukovi?,"[PODFAKULTET HELICERATA, KLASA ARACHNIDA. Pauk..."
8,8,gde se nalazi amurski leopard,[Amurski leopardi imaju najseverniji opseg od ...
9,9,Koliko košta da se sterilizuje moj ženka kunić?,"[Rizik od reproduktivnih karcinoma (jajnika, m..."


In [21]:
df

,id,query,passage_text
0,8,where is the amur leopard found,[Amurski leopardi imaju najseverniji opseg od ...
1,0,Da li ljudska kosa zaustavlja veverice?,[Hrani smo naše veverice u dvorištu tokom jese...
2,7,what class are spiders in,"[PODFAJL CHELICERATA, KLASA ARACHNIDA. Paukovi..."
3,4,šta je hardver u računaru,"[Hardver se najbolje opisuje kao uređaj, poput..."
4,2,what is a apothem,[Apothem. Apotema pravilnog mnogougla je segme...
5,3,average cost for custom canopy,"[Čestitamo! Pronašli ste BuyShade.com, vaš izv..."
6,9,Koliko košta da se sterilizuje moj ženka kunić?,"[Rizik od reproduktivnih karcinoma (jajnika, m..."
7,5,edi logistics definition,[Kao i mnoge druge rane informacione tehnologi...
8,1,what are the benefits of fossil fuels,[Najveća prednost korišćenja fosilnih goriva j...
9,6,why should recreational marijuana be illegal,[Neki ljudi se pitaju 'zašto bi marihuana treb...


In [38]:
# for q in df['explanation']:
#     print(q)

In [36]:
loaded_table = pq.read_table('datasets/naquestions.parquet')
df_natural_language = loaded_table.to_pandas()

In [39]:
df_natural_language

,id,query,passage_text
0,-5501481664893105662,ko je predložio da elektroni pokazuju svojstva...,"[Kroz rad Maks Planka, Alberta Einštajna, Luj ..."
1,8594030243394572667,Koliko senatora ima u Senatu SAD-a?,[Senat Sjedinjenih Američkih Država se sastoji...
2,-716176363900512091,Faza promene iz gasa u čvrsto se naziva,"[Depozicija je termodinamički proces, faza pre..."
3,-5593761806871287545,Koja je najduža engleska reč u rečniku?,[Najduža reč u bilo kom od glavnih engleskih j...
4,2074579308847994444,koliki procenat Zemljine površine je voda,[Voda je raspoređena širom Zemlje. Većina vode...
5,-774521643378360615,Koja je definicija pH vrednosti u vodi?,"[U hemiji, pH (potencijal vodonika) je numerič..."
6,-4418682909598136345,Odakle potiče supa od gnezda ptica?,[Jesti gnezda ptica su gnezda ptica koja stvar...
7,-4809486218828226485,Odakle dolaze neutroni u nuklearnoj fisiji?,[Izvor neutrona za pokretanje je izvor neutron...


In [24]:
df_natural_language

,id,query,passage_text
0,-716176363900512091,Promena faze iz gasa u čvrsto se naziva,"[Depozicija je termodinamički proces, faza pre..."
1,8594030243394572667,Koliko senatora ima u Senatu SAD-a?,[Senat Sjedinjenih Američkih Država se sastoji...
2,2074579308847994444,Koliki procenat Zemljine površine je voda?,[Voda je raspoređena širom Zemlje. Većina vode...
3,-4809486218828226485,Odakle dolaze neutroni u nuklearnoj fisiji?,[Izvor neutrona za pokretanje je izvor neutron...
4,-5501481664893105662,ko je predložio da elektroni pokazuju ponašanj...,"[Kroz rad Maks Planka, Alberta Einštajna, Luj ..."
5,-774521643378360615,Koja je definicija pH vrednosti u vodi?,"[U hemiji, pH (potencijal vodonika) je numerič..."
6,-5593761806871287545,Koja je najduža engleska reč u rečniku?,[Najduža reč u bilo kojem od glavnih engleskih...
7,-4418682909598136345,Odakle potiče supa od gnezda ptica?,[Jesti gnezda ptica su gnezda ptica koja su na...


In [167]:
row_89 = df_natural_language.iloc[89]  # Indexing is 0-based, so 89th row is at index 88
print(row_89)

id                                            8932885154379264842
query                    who rides so late through night and wind
passage_text    [Ko tako kasno jaše kroz noć i vetar? To je ot...
Name: 89, dtype: object


In [25]:
for q in df_natural_language['passage_text']:
    print(q)

['Depozicija je termodinamički proces, faza prelaza u kojoj gas prelazi u čvrsto stanje bez prolaska kroz tečnu fazu. Suprotnost depozicije je sublimacija i zbog toga se ponekad depozicija naziva desublimacija.']
['Senat Sjedinjenih Američkih Država se sastoji od 100 članova, po dva iz svake od 50 saveznih država. U nastavku je lista trenutnih senatora SAD-a koji sede u 115. Kongresu Sjedinjenih Američkih Država.']
['Voda je raspoređena širom Zemlje. Većina vode u Zemljinoj atmosferi i kori potiče iz slane morske vode svetskog okeana, dok slatka voda čini samo 2.5% ukupnog. Zbog okeana koji pokrivaju otprilike 78% površine Zemlje reflektujući plavu svetlost, Zemlja izgleda plavo iz svemira i često se naziva plavom planetom i Bleda Plava Tačka. Procenjuje se da se 1.5 do 11 puta više vode od one u okeanima može naći stotinama milja duboko unutar Zemljinog unutrašnjeg dela, iako ne u tečnom obliku.']
['Izvor neutrona za pokretanje je izvor neutrona koji se koristi za stabilno i pouzdano 

In [91]:
type(df['passage_text'][0])

numpy.ndarray

## Testing GPT Response

In [95]:
response_path = Path("./translation_pipeline_test/naquestions_test_results.jsonl")
assert response_path.exists(), f"File not found: {response_path}"

In [109]:
responses = []
with open(response_path, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        data = json.loads(line)
        request = data[0]
        response = data[1]['choices'][0]['message']['content']
        try:
            response_json = json.loads(response)
            responses.append(response_json)
        except json.JSONDecodeError as e:
            error_data = response
            print(f"Index: {i}, Error: {e}")

Index: 7, Error: Expecting ',' delimiter: line 5 column 1461 (char 1583)


In [111]:
print(error_data)

{
    "translations": [
        {
            "query": "ko ima najviše nagrada za najkorisnijeg igrača All-Star utakmica",
            "passage_text": ["Bob Pettit i Kobe Bryant su jedini igrači koji su osvojili nagradu za najkorisnijeg igrača All-Star utakmica četiri puta. Oscar Robertson, Majkl Džordan, Šekil O'Nil i Lebron Džejms su svaki osvojili nagradu tri puta, dok su Bob Cousy, Džulijus Erving, Azejja Tomas, Međik Džonson, Karl Meloun, Alen Ajverson i Rasel Vestbruk svi osvojili nagradu dva puta. Džejmsova prva nagrada za najkorisnijeg igrača All-Star utakmica 2006. godine učinila ga je najmlađim ikada koji je osvojio nagradu sa 21 godinom i 1 mesecom. Kajri Irving, pobednik nagrade za najkorisnijeg igrača All-Star utakmica 2014. godine, drugi je najmlađi sa 21 godinom i 10 meseci. Oni su značajni jer su najmlađi koji su osvojili nagradu, obojica kao igrači Klivlenda. Četiri puta su postojali zajednički pobednici - Elgin Bejlor i Pettit 1959. godine, Džon Stokton i Meloun 1993.

In [108]:
[{"model": "gpt-3.5-turbo-0125", "response_format": {"type": "json_object"}, "temperature": 0, "messages": [{"role": "system", "content": "\n***TRANSLATION FROM ENGLISH TO SERBIAN***\n\n**GOALS**\n\nYou are a professional translator fluent in English and Serbian. \nYour primary goal is to produce a high-quality, natural-sounding translation from English to Serbian. \nYou are translating texts and questions pertaining to the texts. The translation is intended for dataset creation. Look at the example below:\n\n***TRANSLATION EXAMPLE***\n***ENGLISH***\nquery: 'What is a unicorn?'\npassage_text: 'The unicorn is a legendary creature that has been described since antiquity as a beast with a single large, pointed, spiraling horn projecting from its forehead.'\n\n***SERBIAN TRANSLATION***\nquery: 'Šta je jednorog?'\npassage_text: 'Jednorog je mitsko stvorenje koje se od davnina opisuje kao zver sa jednim velikim, šiljastim, spiralnim rogom koji mu viri iz čela.'\n***END OF TRANSLATION EXAMPLE***\n\nTo translate, follow the steps below:\n   **TRANSLATION INSTRUCTIONS**\n   1. Read and understand the sentence in English.\n   2. When you understand the English sentence, start to translate.\n   3. Pay close attention to both left and right context when you are making translation decisions.  4. Convey the original context, tone and meaning in the Serbian translation.\n   4. Avoid literal translations and ensure the output reads naturally in Serbian.\n   5. The translation must be contextually accurate, fluent, and adhere to the grammatical rules and lexicon of the Serbian language.\n   6. The declination of nouns, adjectives, and pronouns must be correct.\n   7. Make sure to proofread the translated text in Serbian and revise any mistakes. If no revisions are needed, provide the translations as they are.\n\n   **FORMATTING INSTRUCTIONS**\n   1. Strings should be enclosed within single quotation marks ('').\n   2. Use double quotes for strings and escape internal quotes with a backslashes (\\).\n \n   **OUTPUT FORMATTING**\n   - Ensure the output is a valid JSON file, parsable by Python's json.loads().\n   - Ensure consistent JSON formatting as illustrated in the example below:\n\n      **EXAMPLE**\n      {\"translations\" : [{\"query\" : 'This is a query',\n                         \"passage_text\" : ['This is one passage. With another sentence.',\n                                          'This is yet another passage. With yet another sentence.']\n                        }\n                        ]\n      }\n       **END OF EXAMPLE**\n\n   - Strictly follow the structure provided in the example when generating the output.   \n   - Make sure to translate text under both \"query\" and \"passage_text\" keys.\n\n"}, {"role": "user", "content": "{\"query_id\": \"4245798066923223457\", \"query\": \"who has the most all star mvp awards\", \"passage_text\": [\"Bob Pettit and Kobe Bryant are the only two players to win the All - Star Game MVP four times . Oscar Robertson , Michael Jordan , Shaquille O'Neal , and LeBron James have each won the award three times , while Bob Cousy , Julius Erving , Isiah Thomas , Magic Johnson , Karl Malone , Allen Iverson , and Russell Westbrook have all won the award twice . James ' first All - Star MVP in 2006 made him the youngest to have ever won the award at the age of 21 years , 1 month . Kyrie Irving , winner of the 2014 All - Star Game MVP , is the second - youngest at 21 years , 10 months . They are notable as being the two youngest to win the award , both as Cleveland Cavaliers . Four of the games had joint winners -- Elgin Baylor and Pettit in 1959 , John Stockton and Malone in 1993 , O'Neal and Tim Duncan in 2000 , and O'Neal and Bryant in 2009 . O'Neal became the first player in All - Star history to share two MVP awards as well as the first player to win the award with multiple teams . The Los Angeles Lakers have had eleven winners while the Boston Celtics have had eight . Duncan of the U.S. Virgin Islands and Irving of Australia are the only winners not born in the United States . Both Duncan and Irving are American citizens , but are considered `` international '' players by the NBA because they were not born in one of the fifty states or Washington , D.C. No player trained entirely outside the U.S. has won the award ; Irving lived in the U.S. since age two , and Duncan played U.S. college basketball at Wake Forest .\"]}"}]}, {"id": "chatcmpl-9yelcepUFNOTyrdTQR677DyHkQ49b", "object": "chat.completion", "created": 1724243416, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n    \"translations\": [\n        {\n            \"query\": \"ko ima najviše nagrada za najkorisnijeg igrača All-Star utakmica\",\n            \"passage_text\": [\"Bob Pettit i Kobe Bryant su jedini igrači koji su osvojili nagradu za najkorisnijeg igrača All-Star utakmica četiri puta. Oscar Robertson, Majkl Džordan, Šekil O'Nil i Lebron Džejms su svaki osvojili nagradu tri puta, dok su Bob Cousy, Džulijus Erving, Azejja Tomas, Međik Džonson, Karl Meloun, Alen Ajverson i Rasel Vestbruk svi osvojili nagradu dva puta. Džejmsova prva nagrada za najkorisnijeg igrača All-Star utakmica 2006. godine učinila ga je najmlađim ikada koji je osvojio nagradu sa 21 godinom i 1 mesecom. Kajri Irving, pobednik nagrade za najkorisnijeg igrača All-Star utakmica 2014. godine, drugi je najmlađi sa 21 godinom i 10 meseci. Oni su značajni jer su najmlađi koji su osvojili nagradu, obojica kao igrači Klivlenda. Četiri puta su postojali zajednički pobednici - Elgin Bejlor i Pettit 1959. godine, Džon Stokton i Meloun 1993. godine, O'Nil i Tim Dankan 2000. godine, i O'Nil i Brajant 2009. godine. O'Nil je postao prvi igrač u istoriji All-Star utakmica koji je podelio dve MVP nagrade, kao i prvi igrač koji je osvojio nagradu sa više timova. Los Anđeles Lejkersi imaju jedanaest pobednika dok Boston Seltiksi imaju osam. Dankan sa Devičanskih Ostrva i Irving iz Australije su jedini pobednici koji nisu rođeni u Sjedinjenim Američkim Državama. I Dankan i Irving su američki državljani, ali ih NBA smatra \"                                                                                                    ", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 967, "completion_tokens": 667, "total_tokens": 1634}, "system_fingerprint": null}, {"id": "4245798066923223457"}]


NameError: name 'null' is not defined

In [124]:
data_m = load_dataset("microsoft/ms_marco", "v1.1", cache_dir=dir)


In [128]:
data_m['test']['query_id']

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [134]:
len(lala)

58875

In [139]:
aa = Path('datasets/msmarco.parquet')

In [140]:
aa.stem

'msmarco'

In [141]:
returned_dict = {
             "id": [],
             "query": [],
             "passage_text": [],
        }
failed = []
# Open and iterate through the .jsonl file
with open(Path("translation_pipeline_test/msmarco_test_results.jsonl"), 'r') as file:
    for line in file:
        id_ = None
        try:
            data = json.loads(line)
            id_ = data[-1]['id']
            returned_data = data[1]['choices'][0]['message']['content']
            tranlation = json.loads(returned_data) # gpt message i.e. translation in this case
            # print(type(tranlation['passage_text']))
            if isinstance(tranlation['passage_text'], int) or isinstance(tranlation['query'], int) or isinstance(id_, int):
                print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
                print(id_)
            # tranlation = returned_data['translations']
            returned_dict['id'].append(id_)
            returned_dict['query'].append(tranlation['query'])
            returned_dict['passage_text'].append(tranlation['passage_text'])
        except Exception as e:
            failed.append({
                'id': id_,
                'exception': e
            })



In [145]:
def all_items_same_type(lst):
    if not lst:  # Handle the case where the list is empty
        return True
    first_type = type(lst[0])
    print(first_type)
    return all(isinstance(item, first_type) for item in lst)

# Example usage:
if not all_items_same_type(returned_dict['passage_text']):
    print("Not all items are of the same type")
else:
    print("All items are of the same type")

# Example usage:
if not all_items_same_type(returned_dict['id']):
    print("Not all items are of the same type")
else:
    print("All items are of the same type")

# Example usage:
if not all_items_same_type(returned_dict['query']):
    print("Not all items are of the same type")
else:
    print("All items are of the same type")

<class 'list'>
All items are of the same type
<class 'str'>
All items are of the same type
<class 'str'>
All items are of the same type


In [158]:
dataset = pd.DataFrame(returned_dict)
# dataset['id'] = dataset['id'].astype(str)
# dataset['query'] = dataset['query'].astype(str)
dataset['passage_text'] = dataset['passage_text'].apply(lambda x: np.array(x, dtype=str))

dataset.to_parquet(Path("datasets/tralala.parquet"), engine='pyarrow')

ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column passage_text with type object')

In [3]:
loaded_table = pq.read_table('datasets/msmarco_1000.parquet')
df1 = loaded_table.to_pandas()

loaded_table = pq.read_table('datasets/msmarco_2000.parquet')
df2 = loaded_table.to_pandas()


merged_df = pd.concat([df1, df2], ignore_index=True)


In [4]:
len(merged_df)

3873

In [5]:
merged_df


,id,query,passage_text
0,0,Da li ljudska kosa zaustavlja veverice?,[Hranimo naše veverice u dvorištu tokom jeseni...
1,1,Koji su benefiti fosilnih goriva?,[Najveća prednost korišćenja fosilnih goriva j...
2,2,šta je apotema,[Apotema. Apotema pravilnog mnogougla je segme...
3,3,prosečna cena za prilagođeni baldahin,"[Čestitamo! Pronašli ste BuyShade.com, vaš izv..."
4,4,šta je hardver u računaru,"[Hardver se najbolje opisuje kao uređaj, poput..."
...,...,...,...
3868,3996,definicija ekonomske suverenosti,"[Na primer, sugerisano nam je da je proizvodna..."
3869,3997,Šta znači ime Lyndsey?,[Lyndsey je varijanta pravopisa za Lindsay. Im...
3870,3998,plata profesora sa doktoratom,"[Obrazovanje: Doktorat Plata: $36,000 - $122,0..."
3871,3999,šta je radionika testiranje kose,[RADIONIKA. Radionika je metoda dijagnoze i le...


In [6]:
merged_df.to_parquet("datasets/msmarco.parquet", engine="pyarrow", index=False)
